In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

loan_data=pd.read_csv('/Users/neiloza/Downloads/lending-club-loan-data/loan.csv',low_memory=True)


/Users/neiloza/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
entries=len(loan_data.index)
percentile=.4
loan_data_cleaned=loan_data.dropna(thresh=int(percentile*entries),axis=1)


In [3]:
loan_data_cleaned.columns.values

array(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'issue_d',
       'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
       'recoveries', 'collection_recovery_fee', 'last_pymnt_d',
       'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'policy_code', 'application_type',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il

In [4]:
#purpose has about 12 possible values. should I leave it in the feature space?

In [5]:
#loan_data_cleaned.isnull().sum()

In [6]:
#loan_data_cleaned.last_pymnt_d.value_counts()

In [7]:
v1=loan_data_cleaned
v1=v1.drop(['emp_title','title','pymnt_plan','zip_code','out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv','total_rec_prncp', 'total_rec_int', 'total_rec_late_fee','recoveries', 'collection_recovery_fee', 'last_pymnt_d','last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',],axis=1)
#ready_for_split.issue_d=pd.to_datetime(ready_for_split.issue_d,format='%b-%y')
#columns that need to be converted to datetiem: issuw_d,last_pymnt_d, next_pymnt_d, last_credit_pull_d
v1.loan_status.value_counts()
v2=v1[v1.loan_status!='Fully Paid']
v2.loan_status.value_counts()


Current                                                919695
Charged Off                                            261655
Late (31-120 days)                                      21897
In Grace Period                                          8952
Late (16-30 days)                                        3737
Does not meet the credit policy. Status:Fully Paid       1988
Does not meet the credit policy. Status:Charged Off       761
Default                                                    31
Name: loan_status, dtype: int64

In [8]:
v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1218716 entries, 0 to 2260667
Data columns (total 86 columns):
loan_amnt                     1218716 non-null int64
funded_amnt                   1218716 non-null int64
funded_amnt_inv               1218716 non-null float64
term                          1218716 non-null object
int_rate                      1218716 non-null float64
installment                   1218716 non-null float64
grade                         1218716 non-null object
sub_grade                     1218716 non-null object
emp_length                    1126869 non-null object
home_ownership                1218716 non-null object
annual_inc                    1218712 non-null float64
verification_status           1218716 non-null object
issue_d                       1218716 non-null object
loan_status                   1218716 non-null object
purpose                       1218716 non-null object
addr_state                    1218716 non-null object
dti                  

In [9]:
v3=v2
v3['Target']=0
v3.loc[v3.loan_status=='Current','Target']=1
v3.loc[v3.loan_status=='In Grace Period','Target']=1
v3['Target'].value_counts()
y=v3['Target']

/Users/neiloza/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/neiloza/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
loan_data_cleaned=v2
loan_data_cleaned=loan_data_cleaned.drop(['loan_status','issue_d','earliest_cr_line','Target'],axis=1)
loan_data_cleaned=loan_data_cleaned.reset_index()

In [11]:
vars_to_encode=loan_data_cleaned.select_dtypes(include=[object])



In [12]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
vars_to_encode=loan_data_cleaned.select_dtypes(include=[object]).drop('emp_length',axis=1)
#vars_to_encode.loc[vars_to_encode.emp_length=='< 1 year','emp_length']='0 years'
#vars_to_encode.loc[vars_to_encode.emp_length=='10+ years','emp_length']='10 years'
int_labeled=vars_to_encode.apply(le.fit_transform)
ohe=preprocessing.OneHotEncoder(sparse=False)
ohe.fit(int_labeled)
transformed=ohe.transform(int_labeled)
data=pd.DataFrame(transformed)
data

/Users/neiloza/miniconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
7,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
9,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [13]:
loan_data_cleaned=loan_data_cleaned.drop(['term', 'grade', 'sub_grade' ,'emp_length', 'home_ownership','verification_status', 'purpose' ,'addr_state' ,'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag'],axis=1)


In [14]:
loan_data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218716 entries, 0 to 1218715
Data columns (total 71 columns):
index                         1218716 non-null int64
loan_amnt                     1218716 non-null int64
funded_amnt                   1218716 non-null int64
funded_amnt_inv               1218716 non-null float64
int_rate                      1218716 non-null float64
installment                   1218716 non-null float64
annual_inc                    1218712 non-null float64
dti                           1217255 non-null float64
delinq_2yrs                   1218687 non-null float64
inq_last_6mths                1218687 non-null float64
mths_since_last_delinq        590674 non-null float64
open_acc                      1218687 non-null float64
pub_rec                       1218687 non-null float64
revol_bal                     1218716 non-null int64
revol_util                    1217556 non-null float64
total_acc                     1218687 non-null float64
collections_12_m

In [25]:
loan_data_cleaned=loan_data_cleaned.fillna(loan_data_cleaned.median())
X=pd.concat([loan_data_cleaned,data],ignore_index=True,axis=1)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218716 entries, 0 to 1218715
Columns: 199 entries, 0 to 198
dtypes: float64(194), int64(5)
memory usage: 1.8 GB


In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3,random_state=42)


In [31]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.873320295939718